In [71]:
# import de deep learning

# manejo bbdd
import pandas as pd
import numpy as np
import datetime
import re
import warnings
warnings.filterwarnings("ignore")


#gráficas
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
#estadística
import math
from scipy.stats import norm
from scipy import stats #para box-cox entre otros
from scipy.stats import skew

#preprocesamiento
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from sklearn.preprocessing import OneHotEncoder


#modelos y evaluación
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV # Sirve para probar todas las combinaciones de hiperparametros
# grid_search.best_params_ (para ver cual es la combinacion que mejor performance tiene)
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [72]:
churnData = pd.read_csv("Data/Customer-Churn.txt")
churnData2 = pd.read_csv("Data/Customer-Churn.txt")
churnData.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [73]:
churnData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   OnlineSecurity    7043 non-null   object 
 7   OnlineBackup      7043 non-null   object 
 8   DeviceProtection  7043 non-null   object 
 9   TechSupport       7043 non-null   object 
 10  StreamingTV       7043 non-null   object 
 11  StreamingMovies   7043 non-null   object 
 12  Contract          7043 non-null   object 
 13  MonthlyCharges    7043 non-null   float64
 14  TotalCharges      7043 non-null   object 
 15  Churn             7043 non-null   object 
dtypes: float64(1), int64(2), object(13)
memory

In [74]:
churnData["Churn"].value_counts()

Churn
No     5174
Yes    1869
Name: count, dtype: int64

In [75]:
churnData.shape

(7043, 16)

In [76]:
categorical = churnData.select_dtypes("object")
categorical.head()

,gender,Partner,Dependents,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,TotalCharges,Churn
0,Female,Yes,No,No,No,Yes,No,No,No,No,Month-to-month,29.85,No
1,Male,No,No,Yes,Yes,No,Yes,No,No,No,One year,1889.5,No
2,Male,No,No,Yes,Yes,Yes,No,No,No,No,Month-to-month,108.15,Yes
3,Male,No,No,No,Yes,No,Yes,Yes,No,No,One year,1840.75,No
4,Female,No,No,Yes,No,No,No,No,No,No,Month-to-month,151.65,Yes


In [77]:
numerical1 = churnData.select_dtypes("int")
numerical2 = churnData.select_dtypes("float")

numerical = pd.concat([numerical1, numerical2], axis=1)
suma = len(categorical.columns) + len(numerical.columns)
print(suma)


16


In [78]:
numerical.head()

,SeniorCitizen,tenure,MonthlyCharges
0,0,1,29.85
1,0,34,56.95
2,0,2,53.85
3,0,45,42.30
4,0,2,70.70


In [79]:
encoder = OneHotEncoder(drop='first').fit(categorical)
encoded_categorical = encoder.transform(categorical).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical)
X = pd.concat([encoded_categorical, numerical], axis = 1)
X.head()
X.columns = X.columns.astype(str)
X["6548"].value_counts()
X_2 = X.copy()

In [80]:
smote = SMOTE()

y = X['6548']
X = X.drop("6548", axis=1)
X_sm, y_sm = smote.fit_resample(X, y)
y_sm.value_counts()

6548
0.0    5174
1.0    5174
Name: count, dtype: int64

In [81]:


X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=42)

model_logistic = LogisticRegression()

model_logistic.fit(X_train, y_train)

y_pred_LR = model_logistic.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred_LR)}")
print(classification_report(y_test, y_pred_LR))

Accuracy: 0.7869565217391304
              precision    recall  f1-score   support

         0.0       0.81      0.75      0.78      1021
         1.0       0.77      0.83      0.80      1049

    accuracy                           0.79      2070
   macro avg       0.79      0.79      0.79      2070
weighted avg       0.79      0.79      0.79      2070



In [82]:
from sklearn.tree import DecisionTreeClassifier

treeclass = DecisionTreeClassifier()


treeclass.fit(X_train, y_train)

y_pred_DTC = treeclass.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred_DTC)}")
print(classification_report(y_test, y_pred_DTC))

Accuracy: 0.8420289855072464
              precision    recall  f1-score   support

         0.0       0.82      0.87      0.84      1021
         1.0       0.87      0.81      0.84      1049

    accuracy                           0.84      2070
   macro avg       0.84      0.84      0.84      2070
weighted avg       0.84      0.84      0.84      2070



La diferencia entre ambos R² es muy baja, podríamos decir que es prácticamente insignificante. Tenemos una varianza en el R² de 0.071, podríamos decir que los dos modelos tienen un rendimiento similar.

# Segunda parte del lab

In [83]:
churnData2.head(1)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No


In [84]:
churnData_dummies = pd.get_dummies(churnData2, drop_first=True)
churnData_dummies.head()

,SeniorCitizen,tenure,MonthlyCharges,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No internet service,...,TotalCharges_996.45,TotalCharges_996.85,TotalCharges_996.95,TotalCharges_997.65,TotalCharges_997.75,TotalCharges_998.1,TotalCharges_999.45,TotalCharges_999.8,TotalCharges_999.9,Churn_Yes
0,0,1,29.85,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,0,34,56.95,True,False,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,0,2,53.85,True,False,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,True
3,0,45,42.30,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4,0,2,70.70,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [85]:
X = churnData_dummies.drop('Churn_Yes', axis=1)  
y = churnData_dummies['Churn_Yes']


In [86]:
tl = TomekLinks()
X_resampled, y_resampled = tl.fit_resample(X, y)

In [87]:
y.value_counts()

Churn_Yes
False    5174
True     1869
Name: count, dtype: int64

In [88]:
y_resampled.value_counts()


Churn_Yes
False    4715
True     1869
Name: count, dtype: int64

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

model_logistic = LogisticRegression()
model_logistic.fit(X_train, y_train)


y_pred_lr = model_logistic.predict(X_test)

print(f"Accuracy (Logistic Regression): {accuracy_score(y_test, y_pred_lr)}")
print(classification_report(y_test, y_pred_lr))

Accuracy (Logistic Regression): 0.8170083523158694
              precision    recall  f1-score   support

       False       0.86      0.89      0.87       948
        True       0.68      0.64      0.66       369

    accuracy                           0.82      1317
   macro avg       0.77      0.76      0.77      1317
weighted avg       0.81      0.82      0.82      1317



In [90]:
treeclass = DecisionTreeClassifier()
treeclass.fit(X_train, y_train)


y_pred_tree = treeclass.predict(X_test)
print(f"Accuracy (Decision Tree): {accuracy_score(y_test, y_pred_tree)}")
print(classification_report(y_test, y_pred_tree))

Accuracy (Decision Tree): 0.7843583902809416
              precision    recall  f1-score   support

       False       0.84      0.86      0.85       948
        True       0.62      0.59      0.61       369

    accuracy                           0.78      1317
   macro avg       0.73      0.73      0.73      1317
weighted avg       0.78      0.78      0.78      1317



Parece que la regresión lineal predice mejor al hacer el downsampling, lo cual es entendible ya que al eliminar el "ruido" la regresión lineal mejora. Por otro lado, el árbol de decisión sufrió negativamente el downsampling. Supongo que esto se debe a que, cuanta más información tenga el árbol, mejores decisiones podrá tomar.